In [ ]:
#default_exp representation

In [ ]:
#export
from fastai2.vision.all import *

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
path = untar_data(URLs.MNIST_TINY)
dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False)
m = learn.model

# Representation

> Functions and utilities to get representations of pytorch/fastai objects.

## TODO
* recursively obtain representations

## Model representation

First we want to represent any model as a graph. A graph consists of nodes and links, the nodes will be a collection of nn.Module's and nn.Parameter's, and the links will describe the connections between them.

In [ ]:
#export
class Representation:
    'Representation of a Learn object.'
    def __init__(self, data): self.data = data
    def __repr__(self): return f'{self.__class__.__name__} (\n{self.data}\n)'

In [ ]:
#export
class Node:
    "Represents a Module or Parameter."
    def __init__(self, name, idx, typ, obj=None, nodes=None, links=None, xtra=None):
        store_attr(self, 'name,idx,typ,obj')
        self.nodes = ifnone(nodes, [])
        self.links = ifnone(links, [])
        self.xtra  = ifnone(xtra , {})

    def __repr__(self):
        res = f'({self.__class__.__name__}): {self.name}, type: {self.typ}, idx={self.idx}'
        if len(self.xtra): res += f'\nxtra: {self.xtra}'
        if len(self.nodes):
            res += '\nnodes: ['
            for node in self.nodes: res += f'\n  {node}'
            res += '\n]'

        if len(self.links):
            res += '\nlinks: ['
            for link in self.links: res += f'\n  {link}'
            res += '\n]'
            
        return res

In [ ]:
#export
@patch
def to_representation(self:nn.Module, name=None, idx=0, xtra=None):
    "Obtain information of the Module and stores it on a `Node`."
    name = ifnone(name, self.__class__.__name__)
    xtra = ifnone(xtra, {})
    typ = 'Sequential' if isinstance(self, nn.Sequential) else 'Module'
    nodes,links = _get_module_nodes(self)
    if hasattr(self, '_xtra'): xtra.update(self._xtra)
    return Node(name, idx, typ, self, nodes, links, xtra)

def _get_module_nodes(module:nn.Module):
    "Obtain the `Node` representation for all the module childrens."
    nodes,links = [],[]
    is_seq = isinstance(module, nn.Sequential)
    for i,(n,m) in enumerate(module.named_children()):
        if is_seq: n = f'{n}_{m.__class__.__name__}'
        nodes.append(m.to_representation(n,i))
        if i>0: links.append({'source':i-1, 'target':i})

    return nodes,links

In [ ]:
show_doc(nn.Module.to_representation)

<h4 id="Module.to_representation" class="doc_header"><code>Module.to_representation</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>Module.to_representation</code>(**`name`**=*`None`*, **`idx`**=*`0`*, **`xtra`**=*`None`*)

Obtain information of the Module and stores it on a [`Node`](/fastexplorer/representation#Node).

In [ ]:
#export
@patch
def to_representation(self:Learner):
    "Gets a representation of the Learner to be passed to a web client."
    xb,yb = self.dls.train.one_batch()
    def _get_info(m, i, o):
        params,trainable = total_params(m)
        m._xtra = {'params': params, 'trainable': trainable, 'shape': o.shape}
    
    model = self.model
    layers = flatten_model(model)
    with Hooks(layers, _get_info) as h: model.eval()(xb)
    
    nodes = [Node('Input', 0, 'Input', xtra={'shape':list(xb.shape)}),
             self.model.to_representation(xtra={'open': True}),
             Node('Output', 0, 'Output')]
    links = [{'source':i, 'target':i+1} for i in range_of(nodes)]
    rep = Representation(Node('Leaner', 0, 'Learner', nodes=nodes, links=links, xtra={'open': True}))
    _update_shapes(rep.data)
    for layer in layers: del(layer._xtra) # Clean
    nodes[-1].xtra['shape'] = nodes[-2].xtra.get('shape')
    return rep

def _update_shapes(node):
    shape = node.xtra.get('shape')
    childs = node.nodes
    if (shape is None) and len(childs):
        for n in node.nodes: _update_shapes(n)
        node.xtra['shape'] = childs[-1].xtra.get('shape')

In [ ]:
show_doc(Learner.to_representation)

<h4 id="Learner.to_representation" class="doc_header"><code>Learner.to_representation</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>Learner.to_representation</code>()

Gets a representation of the Learner to be passed to a web client.

In [ ]:
r = learn.to_representation()
PrettyString(r.data)[:300]

"(Node): Leaner, type: Learner, idx=0\nxtra: {'open': True, 'shape': None}\nnodes: [\n  (Node): Input, type: Input, idx=0\nxtra: {'shape': [64, 3, 28, 28]}\n  (Node): Sequential, type: Sequential, idx=0\nxtra: {'open': True, 'shape': torch.Size([64, 2])}\nnodes: [\n  (Node): 0_Sequential, type: Sequential, i"

In [ ]:
#hide
xb,yb = learn.dls.train.one_batch()

In [ ]:
#export
@patch
def get_dict(self:Node):
    "Gets the dictionary of the `Node`."
    res = {'name':self.name, 'type':self.typ, 'index':self.idx}
    if len(self.nodes): res['nodes'] = [o.get_dict() for o in self.nodes]
    if len(self.links): res['links'] = self.links
    if len(self.xtra) : res['xtra']  = self.xtra
    return res

In [ ]:
show_doc(Node.get_dict)

<h4 id="Node.get_dict" class="doc_header"><code>Node.get_dict</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>Node.get_dict</code>()

Gets the dictionary of the [`Node`](/fastexplorer/representation#Node).

In [ ]:
#export
@patch
def to_json(self:Representation):
    "Gets the seriable json from the Leaner `Representation`."
    return json.dumps(self.data.get_dict())

In [ ]:
show_doc(Representation.to_json)

<h4 id="Representation.to_json" class="doc_header"><code>Representation.to_json</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>Representation.to_json</code>()

Gets the seriable json from the Leaner [`Representation`](/fastexplorer/representation#Representation).

In [ ]:
r.to_json()[:300]

'{"name": "Leaner", "type": "Learner", "index": 0, "nodes": [{"name": "Input", "type": "Input", "index": 0, "xtra": {"shape": [64, 3, 28, 28]}}, {"name": "Sequential", "type": "Sequential", "index": 0, "nodes": [{"name": "0_Sequential", "type": "Sequential", "index": 0, "nodes": [{"name": "0_Conv2d",'

In [ ]:
#hide
learn.summary()

Sequential (Input shape: ['64 x 3 x 28 x 28'])
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               64 x 64 x 14 x 14    9,408      True      
________________________________________________________________
BatchNorm2d          64 x 64 x 14 x 14    128        True      
________________________________________________________________
ReLU                 64 x 64 x 14 x 14    0          False     
________________________________________________________________
MaxPool2d            64 x 64 x 7 x 7      0          False     
________________________________________________________________
Conv2d               64 x 64 x 7 x 7      36,864     True      
________________________________________________________________
BatchNorm2d          64 x 64 x 7 x 7      128        True      
________________________________________________________________
ReLU                 64 x 64 x 7 x 7      0          False     
___________________________________________________

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_representation.ipynb.
Converted 01_explorer.ipynb.
Converted 10_tutorial.ipynb.
Converted index.ipynb.
